<a href="https://colab.research.google.com/github/Ramprabu95/AI_based_image_compression/blob/main/Learning_convolutional_Networks_for_Content_weighted_Image_Compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learning Convolutional Networks for Content Weighted Image Compression

# **Introduction:**


In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F

In [8]:
# Definition of convolution layer
def conv(ni, nf, kernal_size=3, stride=1, padding=1, **kwargs):
    _conv = nn.Conv2d(ni, nf, kernel_size=kernal_size,stride=stride,padding=padding, **kwargs)
    return _conv

# Definition of a residual block
class ResBlock(nn.Module):
    def __init__(self, ni, nh=128):
        super().__init__()
        self.conv1 = conv(ni, nh)
        self.conv2 = conv(nh, ni)
    def forward(self, x):
        return x  + self.conv2(F.relu(self.conv1(x)))

# Definition of Relu block
relu = nn.ReLU()

# Definition of encoder
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.return_imp_map = return_imp_map
        #
        self.step1 = nn.Sequential(conv(ni=3, nf=128, kernal_size=8, stride=4, padding=2), relu,
                                   ResBlock(nh=128), relu,
                                   conv(ni=128, nf=256, kernal_size=4, stride=2, padding=1), relu,
                                   ResBlock(nh=256), relu,
                                   ResBlock(nh=256), relu)
    def forward(self,x):
        step1 = self.step1(x)
        return(step1)

# Function that does the binarizing action
class Binarizing_function(torch.autograd.Function):
  @staticmethod
  def forward(ctx, i):
    return (i>0.5).float()
  
  @staticmethod
  def backward(ctx,grad_output):
    return grad_output

def binary_value(x):
  return Binarizing_function.apply(x)

class Bin(nn.Module):
  def __init__(self,func):
    super().__init__()
    self.func = func

  def forward(self,x):
    return self.func(x)

# Definition of binarize 
class Binarize(nn.Module):
  def __init__(self):
    super().__init__(self,return_imp_map=False)
    self.step2 = nn.Sequential(conv(ni=256,nf=64,kernal_size=3,stride=1,padding=1),
                               nn.Sigmoid(),
                               Bin(binary_value))
    
